In [1]:
%matplotlib qt
from matplotlib import pyplot as plt
from numpy.random import rand
from skimage.io import imread, imsave
from skimage.util import img_as_float
from skimage.transform import rotate
from skimage.filters import gaussian as ski_gaussian
from pylab import ginput
from scipy.signal import convolve, convolve2d, correlate2d
from scipy.signal import  gaussian as scipy_gaussian
import numpy as np
import os



os.chdir("..\Mats")

1

1.1

In [2]:
def gammaFunc(image,gamma,HSV=False):
    img = np.copy(image)
    
    try:
        for i in range(image.shape[2]):
            img[:,:,i] = 255*(img[:,:,i]/255)**gamma
        return img
    except:
        return 255*(image/255)**gamma

In [5]:
A = imread("cameraman.tif")

B=gammaFunc(A,2)

plt.subplot(1,2,1)
plt.imshow(A,cmap="gray")
plt.subplot(1,2,2)
plt.imshow(B,cmap="gray")
plt.show()

1.2

In [4]:
A = imread("autumn.tif")

B = gammaFunc(np.copy(A),0.5)

plt.subplot(1,2,1)
plt.imshow(A,cmap="gray")
plt.subplot(1,2,2)
plt.imshow(B,cmap="gray")
plt.show()

1.3

In [11]:
A = img_as_float(A)

In [9]:
R = A[:,:,0]
G = A[:,:,1]
B = A[:,:,2]

In [11]:
np.sqrt(R**2+G**2+B**2 - R*B - R*G - G*B)

array([[14.56 , 13.86 , 10.54 , ..., 11.79 ,  6.855,  9.59 ],
       [ 9.75 , 10.77 , 13.6  , ..., 13.38 , 14.87 , 14.87 ],
       [11.49 , 12.125, 10.77 , ..., 11.79 , 10.05 , 13.71 ],
       ...,
       [11.266,  7.81 ,  7.   , ...,  3.   ,  6.246,  2.   ],
       [ 5.566,  4.   ,  1.   , ...,  1.732,  5.   ,  5.293],
       [ 5.566,  2.   ,  2.   , ...,  3.605,  1.   ,  4.   ]],
      dtype=float16)

In [19]:
def gammaFunc(image,gamma,HSV=False):
    img = np.copy(img_as_float(image))
    if HSV == True:
        R = img[:,:,0]
        G = img[:,:,1]
        B = img[:,:,2]
        print(R**2+G**2+B**2 - R*B - R*G - G*B)
        theta = np.arccos(((2*R-B-G)/2) / np.sqrt(R**2+G**2+B**2 - R*B - R*G - G*B))

        return theta
    else:
        try:
            for i in range(image.shape[2]):
                img[:,:,i] = 255*(img[:,:,i]/255)**gamma
            return img
        except:
            return 255*(image/255)**gamma

In [20]:
G = gammaFunc(A,1,HSV=True)

[[7.19723183e-03 6.88965782e-03 5.64398308e-03 ... 1.00115340e-02
  8.59669358e-03 9.28873510e-03]
 [5.39792388e-03 5.72087659e-03 6.78200692e-03 ... 1.06266820e-02
  1.12725875e-02 1.12725875e-02]
 [5.96693579e-03 6.19761630e-03 5.72087659e-03 ... 1.00115340e-02
  9.42714341e-03 1.07650903e-02]
 ...
 [1.95309496e-03 9.38100730e-04 7.53556324e-04 ... 1.38408304e-04
  5.99769319e-04 6.15148020e-05]
 [4.76739715e-04 2.46059208e-04 1.53787005e-05 ... 4.61361015e-05
  3.84467512e-04 4.30603614e-04]
 [4.76739715e-04 6.15148020e-05 6.15148020e-05 ... 1.99923106e-04
  1.53787005e-05 2.46059208e-04]]


C:\Users\zhakk\AppData\Local\Temp\ipykernel_14700\2469995423.py:8: RuntimeWarning: invalid value encountered in sqrt
  theta = np.arccos(((2*R-B-G)/2) / np.sqrt(R**2+G**2+B**2 - R*B - R*G - G*B))
C:\Users\zhakk\AppData\Local\Temp\ipykernel_14700\2469995423.py:8: RuntimeWarning: invalid value encountered in divide
  theta = np.arccos(((2*R-B-G)/2) / np.sqrt(R**2+G**2+B**2 - R*B - R*G - G*B))
C:\Users\zhakk\AppData\Local\Temp\ipykernel_14700\2469995423.py:8: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos(((2*R-B-G)/2) / np.sqrt(R**2+G**2+B**2 - R*B - R*G - G*B))


In [21]:
G

array([[8.04633677e-01, 7.13724379e-01, 6.28188506e-01, ...,
        7.00895077e-01, 6.72179079e-01, 6.87083748e-01],
       [7.66162650e-01, 6.79775647e-01, 6.66946345e-01, ...,
        6.76359386e-01, 6.53237039e-01, 6.53237039e-01],
       [7.80243433e-01, 6.48456806e-01, 6.79775647e-01, ...,
        7.00895077e-01, 7.26945370e-01, 7.13724379e-01],
       ...,
       [2.57361148e+00, 2.55393163e+00, 3.14159236e+00, ...,
                   nan, 7.66162650e-01, 1.04719755e+00],
       [2.98541578e+00, 3.14159220e+00, 2.09439510e+00, ...,
        2.61799388e+00, 2.09439510e+00, 2.42786827e+00],
       [2.98541578e+00,            nan, 1.04719755e+00, ...,
        1.28976143e+00, 1.04719755e+00, 5.96046448e-08]])

In [22]:
index = np.where(np.any(G==np.nan))


In [23]:
print(index)

(array([], dtype=int64),)


In [24]:
plt.imshow(G)